<a href="https://colab.research.google.com/github/TingyuZhao19/MSSP6070/blob/main/WeeklyModules/Week04/PARTICIPATION_ACTIVITY_Week_4_Tingyu_Zhao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import re
from IPython.display import display
import pandas as pd

csv_path = '/content/drive/MyDrive/MSSP607/Working with Pandas DataFrames/parsed.csv'
df = pd.read_csv(csv_path)
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,status,time,title,tsunami,type,types,tz,updated,url,parsed_place
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...,California
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...,California
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...,California
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...,California
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...,California


In [59]:
# a: 95th percentile for Japan with magType 'mb'
mask_japan = df['parsed_place'].astype(str).str.contains(r'\bjapan\b', case=False)
mask_mb    = df['magType'].astype(str).str.lower().eq('mb')
japan_mb   = pd.to_numeric(df.loc[mask_japan & mask_mb, 'mag'], errors='coerce')
p95        = float(japan_mb.quantile(0.95))
print("2(a) Japan, magType='mb', 95th percentile:", p95, f"(n={japan_mb.shape[0]})")

2(a) Japan, magType='mb', 95th percentile: 4.9 (n=50)


In [60]:
# b: the % of Indonesia quakes with tsunamis
mask_indo = df['parsed_place'].astype(str).str.contains(r'\bindonesia\b', case=False)
indo      = df.loc[mask_indo]
num_total = int(indo.shape[0])
tsu_true  = ((df['tsunami'] == 1) | (df['tsunami'] == '1'))
num_tsu   = int(tsu_true.loc[mask_indo].sum())
pct       = (num_tsu / num_total * 100.0)
print(f"2(b) Indonesia: {pct:.3f}% had tsunamis ({num_tsu}/{num_total})")

2(b) Indonesia: 23.129% had tsunamis (34/147)


In [73]:
# c: Get summary statistics for earthquakes in Nevada
earthquake_nevada = df[df['parsed_place'].astype(str).str.contains(r'\bnevada\b', case=False)]
summary_nevada = pd.to_numeric(earthquake_nevada['mag'], errors='coerce').describe()
print(summary_nevada)

count    681.000000
mean       0.500073
std        0.696710
min       -0.500000
25%       -0.100000
50%        0.400000
75%        0.900000
max        2.900000
Name: mag, dtype: float64


In [71]:
# d: Add Ring of Fire flag
tokens_other = [
    'bolivia','chile','ecuador','peru','costa rica','guatemala',
    'japan','philippines','indonesia','new zealand','antarctic','antarctica',
    'canada','fiji','alaska','washington','california','russia','taiwan','tonga',
    'kermadec islands'
]

has_others     = df['parsed_place'].astype(str).str.lower().str.contains('|'.join(tokens_other))
has_mexico     = df['parsed_place'].astype(str).str.lower().str.contains(r'\bmexico\b')
has_new_mexico = df['parsed_place'].astype(str).str.lower().str.contains(r'\bnew\s+mexico\b')

df['ring_of_fire'] = has_others | (has_mexico & ~has_new_mexico)
print("2(d) ring_of_fire counts:\n", df['ring_of_fire'].value_counts())

2(d) ring_of_fire counts:
 ring_of_fire
True     7188
False    2144
Name: count, dtype: int64


In [75]:
# e: Counts on vs off Ring of Fire
counts = df['ring_of_fire'].value_counts().rename(index={True:'on_ring_of_fire', False:'outside'})
counts_df = counts.reset_index().rename(columns={'index':'where','ring_of_fire':'count'})
print("on/off counts:")
display(counts_df)

on/off counts:


,count,count
0,on_ring_of_fire,7188
1,outside,2144


In [64]:
# f: Tsunami count along the Ring of Fire
tsunami_on_ring = int(((df['ring_of_fire'] == True) & ((df['tsunami'] == 1) | (df['tsunami'] == '1'))).sum())
total_on_ring   = int((df['ring_of_fire'] == True).sum())
print(f"2(f) Tsunami count on Ring of Fire: {tsunami_on_ring} (total quakes on ring = {total_on_ring})")

2(f) Tsunami count on Ring of Fire: 45 (total quakes on ring = 7188)
